In [1]:
#!pip install -r requirements.txt

In [18]:
import pandas as pd

In [3]:
df= pd.read_csv('./lyrics_train.csv', encoding='utf-8')
#df_test = pd.read_csv('/content/drive/MyDrive/monografia/tcc/projeto/lyrics_test.csv', encoding='utf-8')
"""
Lexicon source is (C) 2016 National Research Council Canada (NRC) 
and library is for research purposes only.  
Source: http://sentiment.nrc.ca/lexicons-for-research/
"""
lexicon = pd.read_csv('./pt-nrc-lexicon.csv', sep=';', encoding='utf-8')

In [29]:
df['pp_lyric'] = df['lyric'].tail(5).apply(clean_lyrics)
antes = df.loc[6506, 'lyric']
depois = df.loc[6506, 'pp_lyric']
print(f'antes: {antes}')
print(f'depois: {depois}')


antes: Não adianta vir Com esse lero Prá cima de mim Com essa fala mansa Balançando a pança... Eu tomei uma chipoca Prá me distrair Tem rastapé lá no bailão Não deixo de ir... Não quero ver Ninguém ficar parado Ou do lado de fora Arriba Saia já chegou E J.Bernard Está tocando agora...(2x) Lá vem Zé Break Tocando seu xote Pela vida a fora Milto Guerreiro Com seu forró maneiro Já chegou a hora... E não tem conversa O forró tá bom a beça E vamos chamegar Com 40 graus de febre Pulo feito lebre Sinto cheiro de forró... Lá vem Robério Dos teclados Chegando lá de Teixeira E Valzinho do acordeon Que não é brincadeira 500 anos de forró... (Repetir a letra)
depois: adiantar ver lero prá mim falir manso balançar pança tomar chipoca prá distrair rastapé bailão deixar ninguém ficar parar arribar sair chegar jbernard tocar zé break tocar xotar vidar milto guerreiro forró maneiro chegar horar converso forró tá beça ir chamegar grau febre pular fazer lebre sentir cheirar forró robério teclado chegar t

In [28]:
import string
import re
import spacy

nlp = spacy.load('pt_core_news_sm')
#stop_words = spacy.lang.pt.stop_words.STOP_WORDS
  
def clean_lyrics(text):
  txt = ''
  #trocando letras especiais por normais
  #txt = (' ').join(to_unidecode(word) for word in text.split())
  #removendo acordes
  txt = remove_chords(text)
  #removendo tags de reptição 2x, 4x, etc
  txt = txt.replace('[', '(').replace(']', ')')
  txt = re.sub(r'\([\d]x\)', '', txt)
  #removendo palavras de orientação musical
  txt = re.sub(r'\((.*?)\)', '', txt)  
  #removendo pontuacao
  txt = txt.translate(str.maketrans("", "", string.punctuation))
  #caracteres em minusculo   
  txt = txt.lower()
  #lematização e remoção de stop words
  doc = nlp(txt) 
  txt = (' ').join(token.lemma_ for token in doc 
                   if not token.is_stop
                   and token.is_alpha)
    
  return txt

In [5]:
#carregando lista de acordes para remoção dentro das letras
chords_df = pd.read_csv('./chords-only.txt')
chords_dicty = chords_df.to_dict()
chords = set()

for i in chords_dicty['chord']:
    chords.add(chords_dicty['chord'][i].strip())
    
#função para remoção de acordes dentro de um texto
def remove_chords(text):
    txt_list = text.split()
    return (' ').join(
            word.strip()
            for word in txt_list
            if word.strip() not in chords
        )    

In [6]:
import unidecode as und
def to_unidecode(text):
    txt=""
    for j in text.split():
        txt+= und.unidecode(j) + " "
    return txt.strip()

In [7]:
from spacy import displacy
texto = df.loc[0, 'lyric']
texto = (' ').join(palavra for palavra in texto.split() if re.match(r'\w', palavra))
doc = nlp(texto)
displacy.render(doc)

NameError: name 'nlp' is not defined

In [ ]:
# Teste de pre processamento
#df[df['lyric'].str.contains(r"Tráz")]
lyric = df.loc[4879, 'lyric']
print(f'antes: {lyric}')
lyric = clean_lyrics(lyric)
print(f'depois: {lyric}')


antes: Eu cheguei, ela também chegou Espiei, ela me espiou Eu saquei que ela me sacou Eu gostei, sei que ela gostou Eu pisquei, ela também piscou Eu paquerei, ela me paquerou Levantei, ela se alevantou Eu chamei, aí ela num vêi Eu fiquei putin Eu sentei, ela também sentou Disfarcei, ela disfarçou Eu encarei, ela nem olhou Eu pelejei, aí ela encarou Paquerei, ela me paquerou Eu pisquei, ela também piscou Levantei, ela se alevantou Eu chamei, aí ela num vêi Eu fiquei Putin
depois: chegar tambem chegar espiar espiar sacar sacar gostar gostar piscar tambem piscar paquerei paquerou levantar alevantar chamar ai vei ficar putin sentar tambem sentar disfarçar disfarcou encarar olhar pelejar ai encarar paquerei paquerou piscar tambem piscar levantar alevantar chamar ai vei ficar putin
